In [1]:
# Parsing the config file

In [34]:
import json
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [148]:
#filtered_list = filter_results(NameModule("real",lis))

In [199]:
def dob_query_builder(query_date,filtered_list=None):

    if(lis.__len__()):

        query ={
        "query": 
        {
        "bool": 
        {
        "filter": 
        {
        "terms": {"_id": lis}
        },
        "must": [
        {"match" : { "DOB_1" :  query_date  }},
        {"match" : { "DOB_2" :  query_date  }}
        ]
        }
        }
        }
    else:
        query={"query": {"bool" : {"filter" : {"must": [
        {"match" : { "DOB_1" :  query_date  }},
        {"match" : { "DOB_2" :  query_date  }}]}}}}
    return (query)
####################################
def Name_query_builder(query_name,filtered_list=None):
    if(filtered_list.__len__()):
                query = {
                "query": 
                    {
                        "bool": 
                        {
                            "filter": 
                            {
                            "terms": {"_id": lis}
                            },
                        "must": [
                                {"match": { "Name": query_name}}
                                ]
                        }
                    }
                }
    else:
        query={"query": {"match" : {"Name" : {"query" : query_name,"analyzer": "my_analyzer"}}}}
    return query
##################################################
def address_query_builder(query_address,filtered_list=None):
    if(filtered_list.__len__()):
        query ={
        "query": 
        {
        "bool": 
        {
        "filter": 
        {
        "terms": {"_id": filtered_list}
        },
        "must": [
        { "match" : {
        "Address" : {
        "query" : query_address

        }
        }}
        ]
        }
        }
        }
    else:
        query={
        "query": {
        "match" : {
        "Address" : {
        "query" : query_address

        }
        }
        }
        }
    return query
    
#print(NameModule("deepak",lis))
##########################################
def Country_query_builder(filtered_list=None):
    
    if((type(filtered_list)=="list")and  (filtered_list.__len__())):
        query ={
        "query": 
        {
        "bool": 
        {
        "filter": 
        {
        "terms": {"_id": filtered_list}
        },
        "must_not": [
        {"match": {  "Country": "MX"}},
    {"match": {  "Country": "Mexico"}}
        ]
        }
        }
        }
    else:
        query={ "query": {"bool": 
        {"must_not": [
        {"match": {  "Country": "MX"}},
        {"match": {  "Country": "Mexico"}}
        ]}

        }
        }
    return query
###########################################3333
def filter_results(param_query):
    res=es.search(index="sl_list",body=param_query)
    l=[]
    for i in res['hits']['hits']:
        #print(i['_source'])
        l.append(i['_id'])
    return l


In [214]:
def final_query(id_list=None):
    if(id_list.__len__()):
        query ={
        "query": 
        {
        "bool": 
        {
        "filter": 
        {
        "terms": {"_id": id_list}
        }
        }
        }
        }
        final_list=[]
        res=es.search(index="sl_list",body=query)
        for i in res['hits']['hits']:
            #print(i['_source'])
            final_list.append(i['_source'])
        else:
            final_list.append("No Match-Clear")
        return final_list

        

In [216]:
Country_query = Country_query_builder()
lis = filter_results(Country_query)
#final_query(id_list=lis)



In [217]:
#for Country - Direct

#Name_query    = build_query("Name","real")
#DOB_query     = date_query_builder("1981/06/05")
#Address_query = address_query_builder("Blvd. Cuauhtemoc 1711, Oficina 305, Colonia Zona Rio Tijuana, Baja California Mexico")
#SSN_query = build_query("SSN","<ssn_value>")
#lis = filter_results(Country_query)
#filter_results(NameModule("real",lis))

In [272]:
import pandas as pd

In [273]:
blist=pd.read_csv('_data/BankCustList.csv',)

In [274]:
for (i,j) in blist.iterrows():
    d = j.to_dict()
    print(d['Name'])
    

Mohammed Ali Ahmed 
Abdul Faizal Aziz
Saleem A. Ibrahim
Saleem A. Hameed-Ibrahim
Saleem A. Ibrahim
Anita Diaz
Carlos Santos


In [299]:
json.dump(d,open("_data/a.json","w"))

In [276]:
config=json.loads(open("_data/sampleAML.json").read())

In [277]:
next_module = config['FirstModuleToProcess']


In [302]:
interim_id_list = []
interim_id_list_count = interim_id_list.__len__()
next_module

'Country'

In [300]:

while(1):
    if((interim_id_list_count==0) and next_module =="Country"):
        print("Entering S1-B1")
        interim_id_list = filter_results(Country_query_builder(d['Country']))
    elif((interim_id_list_count!=0) and (next_module =="Country")):
        print("Entering S2-B1")
        interim_id_list = filter_results(Country_query_builder(d['Country']),interim_id_list)

    if((interim_id_list_count==0) and next_module =="Address"):
        print("Entering S1-B2")
        interim_id_list = filter_results(address_query_builder(d['Address']))
    elif((interim_id_list_count!=0) and (next_module =="Address")):
        print("Entering S2-B2")
        interim_id_list = filter_results(address_query_builder(d['Address']),interim_id_list)

    if((interim_id_list_count==0) and next_module =="Name"):
        print("Entering S1-B3")
        interim_id_list = filter_results(Name_query_builder(d['Name']))
    elif((interim_id_list_count!=0) and (next_module =="Name")):
        print("Entering S2-B3")
        interim_id_list = filter_results(Name_query_builder(d['Name']),interim_id_list)

    if((interim_id_list_count==0) and next_module =="DOB"):
        print("Entering S1-B4")
        interim_id_list = filter_results(dob_query_builder(d['DOB']))
    elif((interim_id_list_count!=0) and (next_module =="DOB")):
        print("Entering S2-B4")
        interim_id_list = filter_results(dob_query_builder(d['DOB']),interim_id_list)

    if(interim_id_list.__len__()):# ifYes
        verdict =config['Flow'][next_module][0]['IfYes']
        if(verdict in ['Address','DOB','Name','SSN','Country']):
            next_module = verdict
            print("we have hits and is moving to"+next_module)
    else: # if No ; No hits
        verdict = config['Flow'][next_module][0]['IfNo']
        print("we have no hits and the verdict is "+verdict)
        break


#print(verdict)
    
    

KeyboardInterrupt: 

In [297]:
print(interim_id_list)

['AV-_iri5bXXRbztgEGIw', 'AV-_irh8bXXRbztgEGIq', 'AV-_iribbXXRbztgEGIt', 'AV-_iri9bXXRbztgEGIy', 'AV-_irgjbXXRbztgEGIm', 'AV-_irhNbXXRbztgEGIn', 'AV-_iriLbXXRbztgEGIr', 'AV-_iriLbXXRbztgEGIs', 'AV-_iri9bXXRbztgEGIx', 'AV-_irjNbXXRbztgEGIz']
